<div style="border: 2px solid #8A9AD0; margin: 1em 0.2em; padding: 0.5em;">

# Introduction to sequencing with Python (part three)

by 

CC-BY licensed content from the [Galaxy Training Network](https://training.galaxyproject.org/)

**Objectives**

- Understanding Python lists
- Understanding Python dictionaries
- Learnig about dynamic programming

**Objectives**

- Understanding of lists and dictionaries
- Learning about dynamic programming
- Learning about how to translate DNA in Python

**Time Estimation: 1h**
</div>


<p><a href="https://xkcd.com/2483"><a href="https://imgs.xkcd.com/comics/seven.png" rel="noopener noreferrer"><img src="https://imgs.xkcd.com/comics/seven.png" alt="XKCD 2483. " loading="lazy" /></a></a></p>
<p>Preclass prep: Chapters <a href="https://greenteapress.com/thinkpython2/html/thinkpython2009.html">5</a> and <a href="https://greenteapress.com/thinkpython2/html/thinkpython2011.html">7</a> from “Think Python”</p>
<blockquote class="quote" cite="https://langmead-lab.org/teaching-materials">
<p>This material uses examples from notebooks developed by <a href="https://langmead-lab.org/teaching-materials/">Ben Langmead</a></p>
</blockquote>
<h1 id="prep">Prep</h1>
<ol>
<li>Start <a href="https://mybinder.org/v2/gh/jupyterlab/jupyterlab-demo/try.jupyter.org?urlpath=lab">JupyterLab</a></li>
<li>Within JupyterLab start a new Python3 notebook</li>
<li>Open <a href="http://cs1110.cs.cornell.edu/tutor/#mode=edit">this page</a> in a new browser tab</li>
</ol>
<h1 id="lists-dynamic-programming-in-sequence-alignment">Lists: Dynamic programming in sequence alignment</h1>
<h2 id="dynamic-programming-matrix-as-a-2d-list">Dynamic programming matrix as a 2D list</h2>
<p>An excellent way to illustrate the utility of lists is to implement a dynamic programming algorithm for sequence alignment. Suppose we have two sequences that deliberately have different lengths:</p>

\[\texttt{G C T A T A C}&#36;\]
<p>and</p>

\[\texttt{G C G T A T G C}&#36;\]
<p>Let’s represent them as the following matrix where the first character \(\epsilon\) represents an empty string:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
\hline
\epsilon \\
\hline
G\\
\hline
 C\\
 \hline
G\\
\hline
T\\
\hline
A\\
 \hline
T\\
\hline
G\\
\hline
C
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<p>In this matrix, the cells are addressed as shown below. They filled in using the following logic:</p>

\[D[i,j] = min\begin{cases}
          \color{green}{D[i-1,j] + 1} &amp; \\
          \color{blue}{D[i,j-1] + 1} &amp; \\
          \color{red}{D[i-1,j-1] + \delta(x[i-1],y[j-1])}
             \end{cases}\]
<p>where \(\color{green}{green}\) is <em>upper</em> cell, \(\color{blue}{blue}\) is <em>left</em> cell, and \(\color{red}{red}\) is <em>upper-left</em> cell:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
            \hline
          \epsilon &amp;  &amp;  &amp;  &amp;  &amp;  &amp;  &amp;  &amp; \\
          \hline
          G &amp; \\
          \hline
          C &amp;  &amp; &amp; \color{red}{D[2,2]} &amp; \color{green}{D[2,3]}\\
          \hline
          G &amp;  &amp; &amp; \color{blue}{D[3,2]} &amp; D[3,3]\\
          \hline
          T &amp; \\
          \hline
          A &amp; \\
          \hline
          T &amp; \\
          \hline
          G &amp; \\
          \hline
          C &amp;
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<h2 id="initializing-the-matrix">Initializing the matrix</h2>
<p>Let’s initialize the first column and first row of the matrix. Because the distance between a string and an empty string is equal to the length of the
string (e.g., a distance between, say, string \(\texttt{TCG}\) and an empty string is 3) this resulting matrix will look like this:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
            \hline
          \epsilon &amp; 0 &amp; 1 &amp; 2 &amp; 3 &amp; 4 &amp; 5 &amp; 6 &amp; 7\\
          \hline
          G &amp; 1\\
          \hline
          C &amp; 2\\
          \hline
          G &amp; 3\\
          \hline
          T &amp; 4\\
          \hline
          A &amp; 5\\
          \hline
          T &amp; 6\\
          \hline
          G &amp; 7\\
          \hline
          C &amp; 8
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<p>This can be achieved with the following code:</p>


In [ ]:
D = np.zeros((len(x)+1, len(y)+1), dtype=int)
D[0, 1:] = range(1, len(y)+1)
D[1:, 0] = range(1, len(x)+1)

<h2 id="filling-the-matrix">Filling the matrix</h2>
<p>Now we can fill the entire matrix by using two nested loops: one iterating over \(i\) coordinate (sequence \(x\)) and the other iterating over \(j\) coordinate (sequence \(y\)):</p>


In [ ]:
for i in range(1, len(x)+1):
    for j in range(1, len(y)+1):
        delt = 1 if x[i-1] != y[j-1] else 0
        D[i, j] = min(D[i-1, j-1]+delt, D[i-1, j]+1, D[i, j-1]+1)

<p>Let’s start with the cell between \(\texttt{G}\) and \(\texttt{G}\). Recall that:</p>

\[D[i,j] = min\begin{cases}
          \color{green}{D[i-1,j] + 1} &amp; \\
          \color{blue}{D[i,j-1] + 1} &amp; \\
          \color{red}{D[i-1,j-1] + \delta(x[i-1],y[j-1])}
             \end{cases}\]
<p>where \(\delta(x[i-1],y[j-1]) = 0\) if \(x[i-1] = y[j-1]\) and \(1\) otherwise. And now let’s color each of the cells corresponding to each part of the above expression:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
\hline
\epsilon &amp; \color{red}0 &amp; \color{green}1 &amp; 2 &amp; 3 &amp; 4 &amp; 5 &amp; 6 &amp; 7\\
\hline
G &amp; \color{blue}1\\
          \hline
          C &amp; 2\\
          \hline
          G &amp; 3\\
          \hline
          T &amp; 4\\
          \hline
          A &amp; 5\\
          \hline
          T &amp; 6\\
          \hline
          G &amp; 7\\
          \hline
          C &amp; 8
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<p>In this specific case:</p>

\[D[i,j] = min\begin{cases}
          \color{green}{D[i-1,j] + 1}\ or\ 0+0=0 &amp; \\
          \color{blue}{D[i,j-1] + 1}\ or\ 1+1=2 &amp; \\
          \color{red}{D[i-1,j-1] + \delta(x[i-1],y[j-1])}\ or\ 1+1=2
             \end{cases}\]
<p>The minimum of 0, 2, and 2 will be 0, so we are putting zero into that cell:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
\hline
\epsilon &amp; \color{red}0 &amp; \color{green}1 &amp; 2 &amp; 3 &amp; 4 &amp; 5 &amp; 6 &amp; 7\\
\hline
G &amp; \color{blue}1 &amp; \color{red}0\\
          \hline
          C &amp; 2\\
          \hline
          G &amp; 3\\
          \hline
          T &amp; 4\\
          \hline
          A &amp; 5\\
          \hline
          T &amp; 6\\
          \hline
          G &amp; 7\\
          \hline
          C &amp; 8
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<p>Using this logic we can fill the entire matrix like this:</p>

\[\begin{array}{ c | c | c | c | c | c | c}
&amp; \epsilon &amp; G &amp; C &amp; T &amp; A &amp; T &amp; A &amp; C\\
\hline
\epsilon &amp; 0 &amp; 1 &amp; 2 &amp; 3 &amp; 4 &amp; 5 &amp; 6 &amp; 7\\
\hline
                 G &amp; 1 &amp; 0 &amp; 1 &amp; 2 &amp; 3 &amp; 4 &amp; 5 &amp; 6\\
          \hline
                 C &amp; 2 &amp; 1 &amp; 0 &amp; 1 &amp; 2 &amp; 3 &amp; 4 &amp; 5\\
          \hline
                 G &amp; 3 &amp; 2 &amp; 1 &amp; 1 &amp; 2 &amp; 3 &amp; 4 &amp; 5\\
          \hline
                 T &amp; 4 &amp; 3 &amp; 2 &amp; 1 &amp; 2 &amp; 2 &amp; 3 &amp; 4\\
          \hline
                 A &amp; 5 &amp; 4 &amp; 3 &amp; 2 &amp; 1 &amp; 2 &amp; 2 &amp; 3\\
          \hline
                 T &amp; 6 &amp; 5 &amp; 4 &amp; 3 &amp; 2 &amp; 1 &amp; 2 &amp; 3\\
          \hline
                 G &amp; 7 &amp; 6 &amp; 5 &amp; 4 &amp; 3 &amp; 2 &amp; 2 &amp; 3\\
          \hline
                 C &amp; 8 &amp; 7 &amp; 6 &amp; 5 &amp; 4 &amp; 3 &amp; 3 &amp; \color{red}2
\end{array}
\\
\textbf{Note}: sequence\ \texttt{X}\ is\ vertical\ and\ sequence\ \texttt{Y}\ is\ horizontal.\]
<p>The lower rightmost cell highlighted in red is special. It contains the value for the edit distance between the two strings.
The following Python script implements this idea. You can see that it is essentially instantaneous:</p>


In [ ]:
import numpy as np

def edDistDp(x, y):
    """ Calculate edit distance between sequences x and y using
        matrix dynamic programming.  Return matrix and distance. """
    D = np.zeros((len(x)+1, len(y)+1), dtype=int)
    D[0, 1:] = range(1, len(y)+1)
    D[1:, 0] = range(1, len(x)+1)
    for i in range(1, len(x)+1):
        for j in range(1, len(y)+1):
            delt = 1 if x[i-1] != y[j-1] else 0
            D[i, j] = min(D[i-1, j-1]+delt, D[i-1, j]+1, D[i, j-1]+1)
    return D,D[len(x),len(y)]

<p>A graphical representation of the matrix between <code style="color: inherit">GCGTATGCACGC</code> and <code style="color: inherit">GCTATGCCACGC</code> looks like this:</p>
<p><a href="https://i.imgur.com/veMfPFt.png" rel="noopener noreferrer"><img src="https://i.imgur.com/veMfPFt.png" alt="A DP matrix with Seaborn. " loading="lazy" /></a></p>
<p>This image is generated using <a href="https://seaborn.pydata.org/index.html">Seaborn</a> package using matrix directly:</p>


In [ ]:
sns.heatmap(D,annot=True,cmap="crest")

<hr />
<h1 id="dictionaries-translating-sequences">Dictionaries: Translating sequences</h1>
<p>Perhaps the best way to demonstrate the utility of dictionaries is by using DNA-to-Protein translation as an example.</p>
<h2 id="using-dictionaries-to-translate-dna">Using dictionaries to translate DNA</h2>
<p>The following dictionary maps codons to corresponding amino acid translations. In this case, codon is the <em>key</em> and amino acid is the <em>value</em>:</p>


In [ ]:
table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }

<p>Let’s generate a random DNA sequence:</p>


In [ ]:
import random
seq = "".join([random.choice('atcg') for x in range(100)])

In [ ]:
seq

<p>‘agaccgtagcccaagtgcgtttgaatgtggctacttgggaggatttcattgcggtctgtctccgtacttgttattggtcttctttctgcattatgacgca’</p>
<p>To translate this sequence we write a code that uses a <code style="color: inherit">for</code> loop that iterates over the DNA sequence in steps of 3, creating a codon at each iteration. If the codon is less than 3 letters long, the loop is broken. The resulting amino acid is then added to the <code style="color: inherit">translation</code> string:</p>


In [ ]:
translation = ""
for i in range(0, len(seq), 3):
    codon = seq[i:i+3].upper()
    if len(codon) < 3: break
    if codon in table:
        translation += table[codon]
    else:
        translation += "X"

print("Translation:", translation)

<p>Translation: RP_PKCV_MWLLGRISLRSVSVLVIGLLSAL_R</p>
<p>Note that the code uses the <code style="color: inherit">upper()</code> method to ensure the codon is in uppercase since the table dictionary is case-sensitive. Additionally, the code checks if the codon is in the table dictionary and if not, it adds the letter “X” to the translation. This is a common way to represent unknown or stop codons in a protein sequence.</p>


In [ ]:
translation

<p>‘RP_PKCV_MWLLGRISLRSVSVLVIGLLSAL_R’</p>
<p>Now we define a function that would perform translation so that we can reuse it later:</p>


In [ ]:
def translate(seq):
    translation = ''
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    for i in range(0, len(seq), 3):
        codon = seq[i:i+3].upper()
        if len(codon) < 3: break
        if codon in table:
            translation += table[codon]
        else:
            translation += "X"
    return(translation)

In [ ]:
translate(seq)

<p>‘RP_PKCV_MWLLGRISLRSVSVLVIGLLSAL_R’</p>
<h2 id="expanding-to-multiple-phases-frames">Expanding to multiple phases (frames)</h2>
<p>We can further modify the function by adding a <code style="color: inherit">phase</code> parameter that would allow translating in any of the three phases:</p>


In [ ]:
def translate_phase(seq,phase):
    translation = ''
    table = {
        'ATA':'I', 'ATC':'I', 'ATT':'I', 'ATG':'M',
        'ACA':'T', 'ACC':'T', 'ACG':'T', 'ACT':'T',
        'AAC':'N', 'AAT':'N', 'AAA':'K', 'AAG':'K',
        'AGC':'S', 'AGT':'S', 'AGA':'R', 'AGG':'R',
        'CTA':'L', 'CTC':'L', 'CTG':'L', 'CTT':'L',
        'CCA':'P', 'CCC':'P', 'CCG':'P', 'CCT':'P',
        'CAC':'H', 'CAT':'H', 'CAA':'Q', 'CAG':'Q',
        'CGA':'R', 'CGC':'R', 'CGG':'R', 'CGT':'R',
        'GTA':'V', 'GTC':'V', 'GTG':'V', 'GTT':'V',
        'GCA':'A', 'GCC':'A', 'GCG':'A', 'GCT':'A',
        'GAC':'D', 'GAT':'D', 'GAA':'E', 'GAG':'E',
        'GGA':'G', 'GGC':'G', 'GGG':'G', 'GGT':'G',
        'TCA':'S', 'TCC':'S', 'TCG':'S', 'TCT':'S',
        'TTC':'F', 'TTT':'F', 'TTA':'L', 'TTG':'L',
        'TAC':'Y', 'TAT':'Y', 'TAA':'_', 'TAG':'_',
        'TGC':'C', 'TGT':'C', 'TGA':'_', 'TGG':'W',
    }
    assert phase >= 0 and phase <= 3, "Phase parameter can only be set to 0, 1, or 2! You specified {}".format(phase)
    for i in range(phase, len(seq), 3):
        codon = seq[i:i+3].upper()
        if len(codon) < 3: break
        if codon in table:
            translation += table[codon]
        else:
            translation += "X"
    return(translation)

In [ ]:
translate_phase(seq,2)

<p>‘TVAQVRLNVATWEDFIAVCLRTCYWSSFCIMT’</p>
<p>To translate in all six reading frames (three of the “+” strand and three of the “-“ strand) we need to be able to create a reverse complement of the sequence. Let’s write a simple function for that.
The cell below implements a function <code style="color: inherit">revcomp</code> that takes a DNA sequence as input and returns its reverse complement. It works by first reversing the sequence using the slice
notation <code style="color: inherit">seq[::-1]</code>, which returns the sequence in reverse order. Then, the <code style="color: inherit">translate</code> method is used with the <code style="color: inherit">str.maketrans</code> function to replace each occurrence
of ‘a’, ‘t’, ‘g’, ‘c’, ‘A’, ‘T’, ‘G’, and ‘C’ in the reversed sequence with ‘t’, ‘a’, ‘c’, ‘g’, ‘T’, ‘A’, ‘C’, and ‘G’, respectively:</p>


In [ ]:
import string

def revcomp(seq):
    return seq[::-1].translate(str.maketrans('atgcATCG','tagcTACG'))

<p>Now let’s use this function to create translation in all six reading frames. The cell below uses a <code style="color: inherit">for</code> loop that iterates over the range <code style="color: inherit">(0, 3)</code>, representing the different phases
(or starting positions) of the translation. At each iteration, the <code style="color: inherit">translate_phase</code> function is called with the DNA sequence and the current phase, and the resulting protein sequence
is appended to the <code style="color: inherit">translations</code> list along with the <code style="color: inherit">phase</code> and the <code style="color: inherit">strand</code> orientation (+ or -):</p>


In [ ]:
translations = []

for i in range(0,3):
    translations.append((translate_phase(seq,i),str(i),'+'))
    translations.append((translate_phase(revcomp(seq),i),str(i),'-'))

In [ ]:
translations

<p>[(‘RP_PKCV_MWLLGRISLRSVSVLVIGLLSAL_R’, ‘0’, ‘+’),
     (‘SLIIDKQQE_ELATDRRIKWWELRRLKASSRSL’, ‘0’, ‘-‘),
     (‘DRSPSAFECGYLGGFHCGLSPYLLLVFFLHYDA’, ‘1’, ‘+’),
     (‘R__STNNRNKN_PQTGESNGGNYGD_KRVPVAC’, ‘1’, ‘-‘),
     (‘TVAQVRLNVATWEDFIAVCLRTCYWSSFCIMT’, ‘2’, ‘+’),
     (‘ADNRQTTGIRTSHRQANQMVGTTEIESEFP_P’, ‘2’, ‘-‘)]</p>
<h2 id="finding-coordinates-of-continuous-translations">Finding coordinates of continuous translations</h2>
<p>The translation we’ve generated above contains stops (e.g., <code style="color: inherit">_</code> symbols). The actual biologically relevant protein sequences are between stops.
We now need to split translation strings into meaningful peptide sequences and compute their coordinates. Let’s begin by splitting a string on <code style="color: inherit">_</code> and computing the start and end positions of each peptide:</p>


In [ ]:
string = "aadsds_dsds_dsds"

split_indices = []
for i,char in enumerate(string):
    if char == "_":
        split_indices.append(i)
print(split_indices)

<p>[6, 11]</p>
<p>The code above generates a list of split indices for a string. The list contains the indices of the characters in the string that match a specified character (in this case, the underscore <code style="color: inherit">_</code> character).</p>
<p>The <code style="color: inherit">enumerate</code> function is used to loop over the characters in the string, and at each iteration, the current index and character are stored in the variables <code style="color: inherit">i</code> and <code style="color: inherit">char</code>, respectively.
If the current character matches the specified character, the index <code style="color: inherit">i</code> is appended to the <code style="color: inherit">split_indices</code> list.</p>
<p>After the loop, the <code style="color: inherit">split_indices</code> list is printed to the console. For the input string <code style="color: inherit">"aadsds_dsds_dsds"</code>, the output would be <code style="color: inherit">[6, 11]</code>, indicating that the dashes are located at indices 6 and 11.</p>
<p>But we actually need coordinates of peptides bound by <code style="color: inherit">_</code> characters. To get to this let’s first modify <code style="color: inherit">split_indices</code> by adding the beginning and end:</p>


In [ ]:
string = "aadsds_dsds_dsds"

split_indices = []
for i,char in enumerate(string):
    if char == "_":
        split_indices.append(i)

split_indices.insert(0,-1)
split_indices.append(len(string))
print(split_indices)

<p>[-1, 6, 11, 16]</p>
<p>Now, let’s convert these to ranges and also stick the peptide sequence in:</p>


In [ ]:
string = "aadsds_dsds_dsds"

split_indices = []
for i, char in enumerate(string):
    if char == "_":
        split_indices.append(i)

split_indices.insert(0, -1)
split_indices.append(len(string))

orfs = string.split('_')

parts = []
for i in range(len(split_indices)-1):
    parts.append((orfs[i],split_indices[i]+1, split_indices[i+1]))

print(parts)

<p>[(‘aadsds’, 0, 6), (‘dsds’, 7, 11), (‘dsds’, 12, 16)]</p>
<p>Now let’s convert this to function:</p>


In [ ]:
def extract_coords(translation):
    split_indices = []
    for i, char in enumerate(translation):
        if char == "_":
            split_indices.append(i)

    split_indices.insert(0, -1)
    split_indices.append(len(translation))

    parts = []
    for i in range(len(split_indices)-1):
        parts.append((translation.split('_')[i], split_indices[i] + 1, split_indices[i + 1]))

    return(parts)

In [ ]:
extract_coords(string)

<p>[(‘aadsds’, 0, 6), (‘dsds’, 7, 11), (‘dsds’, 12, 16)]</p>
<p>And specify the right parameters to make it truly generic:</p>


In [ ]:
def extract_coords_with_annotation(separator,translation,phase,strand):
    split_indices = []
    for i,char in enumerate(translation):
        if char == separator:
            split_indices.append(i)

    split_indices.insert(0,-1)
    split_indices.append(len(translation))

    parts = []
    for i in range(len(split_indices)-1):
        parts.append((translation.split(separator)[i], phase, strand, split_indices[i]+1, split_indices[i+1]))

    return(parts)

In [ ]:
extract_coords_with_annotation('_', string, '0', '+')

<p>[(‘aadsds’, ‘0’, ‘+’, 0, 6),
     (‘dsds’, ‘0’, ‘+’, 7, 11),
     (‘dsds’, ‘0’, ‘+’, 12, 16)]</p>
<h2 id="analyzing-all-translations-of-a-given-sequence">Analyzing all translations of a given sequence</h2>
<p>We begin by parsing the <code style="color: inherit">translations</code> list we defined above:</p>


In [ ]:
all_translations = []
for item in translations:
    all_translations.append(extract_coords_with_annotation('_',item[0],item[1],item[2]))

In [ ]:
all_translations

<p>[[(‘RP’, ‘0’, ‘+’, 0, 2),
      (‘PKCV’, ‘0’, ‘+’, 3, 7),
      (‘MWLLGRISLRSVSVLVIGLLSAL’, ‘0’, ‘+’, 8, 31),
      (‘R’, ‘0’, ‘+’, 32, 33)],
     [(‘SLIIDKQQE’, ‘0’, ‘-‘, 0, 9),
      (‘ELATDRRIKWWELRRLKASSRSL’, ‘0’, ‘-‘, 10, 33)],
     [(‘DRSPSAFECGYLGGFHCGLSPYLLLVFFLHYDA’, ‘1’, ‘+’, 0, 33)],
     [(‘R’, ‘1’, ‘-‘, 0, 1),
      (‘’, ‘1’, ‘-‘, 2, 2),
      (‘STNNRNKN’, ‘1’, ‘-‘, 3, 11),
      (‘PQTGESNGGNYGD’, ‘1’, ‘-‘, 12, 25),
      (‘KRVPVAC’, ‘1’, ‘-‘, 26, 33)],
     [(‘TVAQVRLNVATWEDFIAVCLRTCYWSSFCIMT’, ‘2’, ‘+’, 0, 32)],
     [(‘ADNRQTTGIRTSHRQANQMVGTTEIESEFP’, ‘2’, ‘-‘, 0, 30),
      (‘P’, ‘2’, ‘-‘, 31, 32)]]</p>
<p>Now the problem with this list is that it is nested. However, we need to make it flat:</p>


In [ ]:
flat_list = []
for sublist in all_translations:
    for item in sublist:
        flat_list.append(item)

In [ ]:
flat_list

<p>[(‘RP’, ‘0’, ‘+’, 0, 2),
     (‘PKCV’, ‘0’, ‘+’, 3, 7),
     (‘MWLLGRISLRSVSVLVIGLLSAL’, ‘0’, ‘+’, 8, 31),
     (‘R’, ‘0’, ‘+’, 32, 33),
     (‘SLIIDKQQE’, ‘0’, ‘-‘, 0, 9),
     (‘ELATDRRIKWWELRRLKASSRSL’, ‘0’, ‘-‘, 10, 33),
     (‘DRSPSAFECGYLGGFHCGLSPYLLLVFFLHYDA’, ‘1’, ‘+’, 0, 33),
     (‘R’, ‘1’, ‘-‘, 0, 1),
     (‘’, ‘1’, ‘-‘, 2, 2),
     (‘STNNRNKN’, ‘1’, ‘-‘, 3, 11),
     (‘PQTGESNGGNYGD’, ‘1’, ‘-‘, 12, 25),
     (‘KRVPVAC’, ‘1’, ‘-‘, 26, 33),
     (‘TVAQVRLNVATWEDFIAVCLRTCYWSSFCIMT’, ‘2’, ‘+’, 0, 32),
     (‘ADNRQTTGIRTSHRQANQMVGTTEIESEFP’, ‘2’, ‘-‘, 0, 30),
     (‘P’, ‘2’, ‘-‘, 31, 32)]</p>
<p>Now we can load the list into <code style="color: inherit">Pandas</code> and plot away:</p>


In [ ]:
import pandas as pd
df = pd.DataFrame(flat_list,columns=['aa','frame','phase','start','end'])

In [ ]:
df

<div>
<style scoped="">
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
<thead>
<tr style="text-align: right;">
<th></th>
<th>aa</th>
<th>frame</th>
<th>phase</th>
<th>start</th>
<th>end</th>
</tr>
</thead>
<tbody>
<tr>
<th>0</th>
<td>RP</td>
<td>0</td>
<td>+</td>
<td>0</td>
<td>2</td>
</tr>
<tr>
<th>1</th>
<td>PKCV</td>
<td>0</td>
<td>+</td>
<td>3</td>
<td>7</td>
</tr>
<tr>
<th>2</th>
<td>MWLLGRISLRSVSVLVIGLLSAL</td>
<td>0</td>
<td>+</td>
<td>8</td>
<td>31</td>
</tr>
<tr>
<th>3</th>
<td>R</td>
<td>0</td>
<td>+</td>
<td>32</td>
<td>33</td>
</tr>
<tr>
<th>4</th>
<td>SLIIDKQQE</td>
<td>0</td>
<td>-</td>
<td>0</td>
<td>9</td>
</tr>
<tr>
<th>5</th>
<td>ELATDRRIKWWELRRLKASSRSL</td>
<td>0</td>
<td>-</td>
<td>10</td>
<td>33</td>
</tr>
<tr>
<th>6</th>
<td>DRSPSAFECGYLGGFHCGLSPYLLLVFFLHYDA</td>
<td>1</td>
<td>+</td>
<td>0</td>
<td>33</td>
</tr>
<tr>
<th>7</th>
<td>R</td>
<td>1</td>
<td>-</td>
<td>0</td>
<td>1</td>
</tr>
<tr>
<th>8</th>
<td></td>
<td>1</td>
<td>-</td>
<td>2</td>
<td>2</td>
</tr>
<tr>
<th>9</th>
<td>STNNRNKN</td>
<td>1</td>
<td>-</td>
<td>3</td>
<td>11</td>
</tr>
<tr>
<th>10</th>
<td>PQTGESNGGNYGD</td>
<td>1</td>
<td>-</td>
<td>12</td>
<td>25</td>
</tr>
<tr>
<th>11</th>
<td>KRVPVAC</td>
<td>1</td>
<td>-</td>
<td>26</td>
<td>33</td>
</tr>
<tr>
<th>12</th>
<td>TVAQVRLNVATWEDFIAVCLRTCYWSSFCIMT</td>
<td>2</td>
<td>+</td>
<td>0</td>
<td>32</td>
</tr>
<tr>
<th>13</th>
<td>ADNRQTTGIRTSHRQANQMVGTTEIESEFP</td>
<td>2</td>
<td>-</td>
<td>0</td>
<td>30</td>
</tr>
<tr>
<th>14</th>
<td>P</td>
<td>2</td>
<td>-</td>
<td>31</td>
<td>32</td>
</tr>
</tbody>
</table>
</div>
<p>Now let’s plot it:</p>


In [ ]:
import altair as alt

plus = alt.Chart(df[df['phase']=='+']).mark_rect().encode(
    x = alt.X('start:Q'),
    x2 = alt.X2('end:Q'),
    y = alt.Y('frame:N'),
    color='frame',
    tooltip='aa:N'
             ).properties(
    width=600,
    height=100)

minus = alt.Chart(df[df['phase']=='-']).mark_rect().encode(
    x = alt.X('start:Q',sort=alt.EncodingSortField('start:Q', order='descending')),
    x2 = alt.X2('end:Q'),
    y = alt.Y('frame:N'),
    color='frame',
    tooltip='aa:N'
             ).properties(
    width=600,
    height=100)

plus & minus

<figure id="figure-1" style="max-width: 90%;"><div style="overflow-x: auto"><object data="./images/orfs.svg" type="image/svg+xml" alt="ORFS. ">ORFS.</object></div><a target="_blank" href="./images/orfs.svg" rel="noopener noreferrer"><small>Open image in new tab</small></a><br /><br /><figcaption><span class="figcaption-prefix"><strong>Figure 1</strong>:</span> Translation on plus (top) and minus (bottom) strands.</figcaption></figure>


# Key Points

- Lists and dictionaries are key data structures in Python

# Congratulations on successfully completing this tutorial!

Please [fill out the feedback on the GTN website](https://training.galaxyproject.org/training-material/topics/data-science/tutorials/gnmx-lecture4/tutorial.html#feedback) and check there for further resources!
